# global imports and data

In [1]:
import numpy as np
import math
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, TimeDistributed, Dropout


SEQ_LENGTH=300
HIDDEN_DIM=128
LAYER_NUM=2
BATCH_SIZE=10
GENERATE_LENGTH=100
EPOCHS=10
CAP_VALUE=40000
OFFSET=20000

Using TensorFlow backend.


# load dataset

In [2]:
text=open("datasource/iwa95.txt","r").read()
text=text.lower()[OFFSET:OFFSET+CAP_VALUE]
chars=list(set(text))
VOCAB_SIZE=len(chars)
VOCAB_SIZE
print(len(text))

40000


# map characters

In [3]:
ix_to_char = {ix:char for ix, char in enumerate(chars)}
char_to_ix = {char:ix for ix, char in enumerate(chars)}

# create input and goal sequences

In [4]:
X = np.zeros((len(text), SEQ_LENGTH, VOCAB_SIZE))
y = np.zeros((len(text), SEQ_LENGTH, VOCAB_SIZE))

for i in range(0, len(text)-SEQ_LENGTH):
    X_sequence = text[i:i+SEQ_LENGTH]
    X_sequence_ix = [char_to_ix[value] for value in X_sequence]
    input_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        input_sequence[j][X_sequence_ix[j]] = 1.
    X[i] = input_sequence

    y_sequence = text[i+1:(i+1)+SEQ_LENGTH+1]
    y_sequence_ix = [char_to_ix[value] for value in y_sequence]
    target_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        target_sequence[j][y_sequence_ix[j]] = 1.
    y[i] = target_sequence
    

# generate model

In [5]:
model = Sequential()
model.add(LSTM(HIDDEN_DIM, input_shape=(None, VOCAB_SIZE), return_sequences=True))
model.add(Dropout(0.2))
for i in range(LAYER_NUM - 1):
    model.add(LSTM(HIDDEN_DIM, return_sequences=True))
model.add(TimeDistributed(Dense(VOCAB_SIZE)))
model.add(Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

# train of the network

In [6]:
def generate_text(model, length):
    ix = [np.random.randint(VOCAB_SIZE)]
    y_char = [ix_to_char[ix[-1]]]
    X = np.zeros((1, length, VOCAB_SIZE))
    for i in range(length):
        X[0, i, :][ix[-1]] = 1
        print(ix_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(ix_to_char[ix[-1]])
    return ('').join(y_char)

nb_epoch = 0
while (nb_epoch<EPOCHS):
    print('\n\n')
    model.fit(X, y, batch_size=BATCH_SIZE, verbose=1, nb_epoch=1)
    nb_epoch += 1
    #if nb_epoch % 10 == 0:
    generate_text(model, GENERATE_LENGTH)
        #model.save_weights('checkpoint_{}_epoch_{}.hdf5'.format(HIDDEN_DIM, nb_epoch))

D:\anaconda\envs\TensGPU\lib\site-packages\ipykernel_launcher.py:15: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  from ipykernel import kernelapp as app


Epoch 1/1
40000/40000 [==============================] - 1147s 29ms/step - loss: 1.2547
re actions a plinager heploy, with ibm cloud private and lonigom and products and canstall to maths 


Epoch 1/1
40000/40000 [==============================] - 917s 23ms/step - loss: 0.3082
query secustinal information. ibm workload automation with docker compose
	|	the deployment of ibm w


Epoch 1/1
40000/40000 [==============================] - 912s 23ms/step - loss: 0.1695
1
	|	v “setting alerts or actions if an operation does not start or |	complete
|	in the jtopts state


Epoch 1/1
24470/40000 [=================>............] - ETA: 7:19 - loss: 0.1340

MemoryError: 

In [ ]:
generate_text(model,500)

In [ ]:
def predictText(seed, model, length):
    initial_length=len(seed)
    X_sequence = seed
    X_sequence_ix = [char_to_ix[value] for value in X_sequence]
    X = np.zeros((1, len(seed)+length, VOCAB_SIZE))
    for j in range(initial_length):
        X[0, j, :][X_sequence_ix[j]]= 1.
    
    y_char=[];
    ix=X_sequence_ix[-1]
    for i in range(length):
        X[0, i+initial_length, :][ix] = 1
        print(ix_to_char[ix], end="")
        ix = np.argmax(model.predict(X[:, :initial_length+i+1, :])[0], 1)[-1]
        y_char.append(ix_to_char[ix])
    return ('').join(y_char)

predictText("fo", model, 1000)